# Poppy Universe – Layer 3: Star Matrix Model

Welcome to the **Poppy Universe Layer 3 – Star Matrix notebook**!  
The star dataset is already fully correct. Here, we focus on **building a matrix-based recommendation model** using simulated user interactions and star types. This is a **sandbox environment** to test collaborative filtering before the engine consumes it.

> Note: This notebook currently uses **simulated user interactions** to test the Star matrix.  
> Once we have enough real interactions, the same pipeline will process actual user data for production recommendations.

---

## Goals

1. **Prepare interaction data for matrix factorization**  
   - Map users to star types  
   - Include weighted interactions (views, clicks, favorites, ratings)  
   - Normalize scores for ML input

2. **Build the User × Star_Type matrix**  
   - Users in rows, star types in columns  
   - Populate with interaction strengths  

3. **Perform matrix factorization / prediction**  
   - Generate predicted scores for each user × star_type  
   - Save intermediate CSV for engine integration

4. **Analyze results**  
   - Identify top star types per user  
   - Visualize patterns across users and star types

---

## Folder & File References

- **../../Input_Data/Stars.csv** → Star dataset  
- **../../Input_Data/Simulated_User_Interactions.csv** → User interaction dataset  
- **../../Output_Data/Layer3_Star_Predictions.csv** → Final predictions for engine  
- **Plots/** → Optional heatmaps or visualizations

---

> Note: This notebook focuses **on the star component** of Layer 3. Planets and moons will have separate notebooks, then merged later.


## 0) Imports

In [15]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD

## 1) Load Data

In [16]:
# --- Load interaction dataset ---
# 'backend_df' is injected via papermill by the master notebook if backend data passed the checks
try:
    interactions = backend_df
    print("Using backend-provided interactions")
except NameError:
    # fallback to CSV if running standalone
    interactions = pd.read_csv("../../Input_Data/Semantic_Type_Interactions.csv")
    print("Using simulated CSV interactions")

# Ensure Timestamp is datetime
interactions['Timestamp'] = pd.to_datetime(interactions['Timestamp'])

# Preview
interactions.head()

Using simulated CSV interactions


,Interaction_ID,User_ID,Category_Type,Category_Value,Strength,Timestamp
0,1,41,Planet_Type,Terrestrial,4,2025-11-09 09:01:12.133685
1,2,45,Planet_Type,Gas Giant,2,2025-11-21 20:35:00.693508
2,3,96,Planet_Type,Gas Giant,3,2025-11-14 13:07:48.918870
3,4,90,Moon_Parent,Pluto,5,2025-11-21 22:10:11.030732
4,5,73,Star_Type,A,5,2025-11-23 02:14:58.992412


**Explanation:**  
We’re loading the simulated user × type interaction data to see what we have. The key columns are:
 
- `Interaction_ID`: unique identifier for each interaction  
- `User_ID`: the user who performed the interaction  
- `Category_Type`: the type of category the interaction belongs to (e.g., Star_Type, Planet_Type, Moon_Parent)  
- `Category_Value`: the specific value within the category (e.g., G for Star_Type, Dwarf Planet for Planet_Type)  
- `Strength`: numerical interaction strength (1–5), used as a matrix factorization target  
- `Timestamp`: when the interaction occurred  
 
This gives us the base data we’ll use to compute features like user-type preferences, recency-weighted strengths, and the semantic matrices for the third layer of the recommendation engine.

## 2) Filter out planet and moon data

In [17]:
# Keep only rows where Category_Type is "Star_Type"
star_interactions = interactions[interactions['Category_Type'] == 'Star_Type']

star_interactions.head()

,Interaction_ID,User_ID,Category_Type,Category_Value,Strength,Timestamp
4,5,73,Star_Type,A,5,2025-11-23 02:14:58.992412
6,7,50,Star_Type,F,3,2025-11-23 22:04:20.186015
8,9,55,Star_Type,A,3,2025-11-20 12:44:32.106978
10,11,1,Star_Type,K,3,2025-11-21 14:02:17.096557
13,14,83,Star_Type,A,3,2025-11-15 03:13:02.504632


## 3) Create User × Category Matrix

In [18]:
# Pivot: rows = users, cols = category values, values = max strength (or sum/mean if multiple)
user_category_matrix = star_interactions.pivot_table(
    index='User_ID', 
    columns='Category_Value', 
    values='Strength', 
    aggfunc='max',   # could also be sum or mean
    fill_value=0     # fills missing interactions with 0
)

# Optional: reset column names if you want a flat DataFrame
user_category_matrix = user_category_matrix.reset_index()

print(user_category_matrix.head())


Category_Value  User_ID  A  B  F  G  K  M  O
0                     1  4  5  0  5  5  5  5
1                     2  5  5  5  5  5  4  5
2                     3  5  5  5  5  5  5  5
3                     4  5  4  5  5  5  5  4
4                     5  5  5  5  5  5  4  4


## 4) Matrix Factorization with SGD

In [19]:
# Convert pivot table to numpy array (exclude User_ID column)
R = user_category_matrix.drop('User_ID', axis=1).values
num_users, num_items = R.shape
K = 3  # number of latent features

# Initialize user and item latent matrices randomly
np.random.seed(42)
U = np.random.rand(num_users, K)  # Users × Features
V = np.random.rand(num_items, K)  # Items × Features

# Hyperparameters
alpha = 0.01   # learning rate
beta = 0.02    # regularization term
iterations = 1000

# SGD loop
for it in range(iterations):
    for i in range(num_users):
        for j in range(num_items):
            if R[i, j] > 0:  # only consider observed interactions
                # Predict
                pred = U[i, :].dot(V[j, :].T)
                # Error
                e_ij = R[i, j] - pred
                # Update latent features
                U[i, :] += alpha * (2 * e_ij * V[j, :] - beta * U[i, :])
                V[j, :] += alpha * (2 * e_ij * U[i, :] - beta * V[j, :])

# Reconstruct approximate matrix
R_hat = U.dot(V.T)

print("Original matrix:\n", R)
print("Approximated matrix:\n", R_hat)

Original matrix:
 [[4 5 0 5 5 5 5]
 [5 5 5 5 5 4 5]
 [5 5 5 5 5 5 5]
 [5 4 5 5 5 5 4]
 [5 5 5 5 5 4 4]
 [5 5 4 5 5 5 5]
 [5 0 4 1 3 5 5]
 [4 0 5 5 5 5 5]
 [5 5 5 5 5 5 5]
 [5 5 0 5 5 5 5]
 [5 0 4 5 5 0 5]
 [4 5 5 5 5 4 5]
 [5 0 4 5 2 5 3]
 [4 4 5 4 5 5 5]
 [3 4 5 5 4 5 5]
 [3 5 5 5 5 5 5]
 [4 4 5 5 4 5 3]
 [5 4 5 5 5 5 4]
 [1 5 3 4 5 5 5]
 [4 4 5 4 5 5 0]
 [5 4 4 5 3 5 5]
 [5 5 5 5 5 5 5]
 [3 3 3 3 5 4 0]
 [5 5 5 5 5 5 3]
 [4 5 4 5 5 4 5]
 [5 5 5 5 5 2 5]
 [4 3 2 0 5 5 5]
 [0 5 4 5 4 0 3]
 [5 3 5 5 5 4 3]
 [5 4 4 5 4 5 5]
 [5 5 5 0 5 5 5]
 [4 5 5 5 5 5 5]
 [2 5 2 5 4 5 5]
 [0 5 4 4 5 1 4]
 [4 4 5 4 5 5 5]
 [5 5 0 5 5 5 0]
 [5 5 5 5 3 5 5]
 [5 4 3 0 5 5 0]
 [5 4 5 5 3 5 5]
 [5 5 5 5 4 5 5]
 [5 5 5 0 4 5 4]
 [5 5 5 5 5 5 2]
 [5 5 5 5 3 5 4]
 [4 5 5 5 3 4 4]
 [5 5 0 5 5 4 4]
 [5 5 5 5 5 5 5]
 [4 5 5 5 5 5 4]
 [5 5 5 5 5 5 5]
 [3 5 5 5 5 5 5]
 [5 5 5 5 4 5 2]
 [5 5 5 4 5 5 5]
 [5 5 4 5 5 5 4]
 [4 5 4 5 5 5 4]
 [3 4 5 5 4 5 4]
 [5 5 0 0 3 4 1]
 [5 5 5 5 5 5 5]
 [5 4 4 5 5 2 4]
 [5 5 5 5 5 5

## 5) Convert Approximated Matrix Back to DataFrame

In [20]:
# Convert R_hat back to DataFrame
R_hat_df = pd.DataFrame(R_hat, columns=user_category_matrix.columns[1:])  # skip User_ID
R_hat_df['User_ID'] = user_category_matrix['User_ID'].values
# Optional: reorder columns so User_ID is first
cols = ['User_ID'] + [c for c in R_hat_df.columns if c != 'User_ID']
R_hat_df = R_hat_df[cols]

R_hat_df.head()


Category_Value,User_ID,A,B,F,G,K,M,O
0,1,4.497129,4.922089,4.807713,4.891604,4.655937,5.051348,5.057187
1,2,4.935688,4.951473,5.152167,4.825430,5.011024,4.053328,4.980935
2,3,4.825786,5.051594,5.115339,5.090629,4.927838,4.945876,4.977943
3,4,4.813962,4.669691,5.082049,5.120726,4.761617,4.887453,3.851400
4,5,4.991626,4.684849,5.191058,4.909058,4.934418,4.038963,4.075054


## 6) Save Predicted Matrix to CSV

In [21]:
# Save as CSV for master notebook
R_hat_df.to_csv('Files/Layer3_Star_Predictions.csv', index=False)

# Optional: preview
print(R_hat_df.head())

Category_Value  User_ID         A         B         F         G         K  \
0                     1  4.497129  4.922089  4.807713  4.891604  4.655937   
1                     2  4.935688  4.951473  5.152167  4.825430  5.011024   
2                     3  4.825786  5.051594  5.115339  5.090629  4.927838   
3                     4  4.813962  4.669691  5.082049  5.120726  4.761617   
4                     5  4.991626  4.684849  5.191058  4.909058  4.934418   

Category_Value         M         O  
0               5.051348  5.057187  
1               4.053328  4.980935  
2               4.945876  4.977943  
3               4.887453  3.851400  
4               4.038963  4.075054  
